In [1]:
import os
from biopandas.pdb import PandasPdb
import copy
from rdkit import Chem
from rdkit.Chem import AllChem


In [2]:
print("oi")

oi


In [3]:
def split_file(pathName):
    ppdb = PandasPdb()
    ppdb.read_pdb(pathName)

    atoms = ppdb.df['ATOM']
    hetatms = ppdb.df['HETATM']

    hetatms_split = hetatms.groupby("residue_name")
                        
    return atoms, hetatms_split

In [4]:
def save_mol_as_pdb(mol, output_file):
    # Convert the Mol object to a PDB block
    pdb_block = Chem.MolToPDBBlock(mol)

    # Write the PDB block to the output file
    with open(output_file, 'w') as file:
        file.write(pdb_block)

In [5]:
def add_hydrogens(hetatms_df):
    ppdb = PandasPdb()
    ppdb.df['HETATM'] = hetatms_df
    tmp_file: str = "tmp.pdb"

    ppdb.to_pdb(path=tmp_file, records=None, gz=False, append_newline=True)

    with open(tmp_file, "r") as f:
        file_content = f.read()
        mol = Chem.MolFromPDBBlock(file_content)

    mol_with_hydrogens = Chem.rdmolops.AddHs(mol, addCoords=True, addResidueInfo=True)
    os.remove(tmp_file)

    return mol_with_hydrogens

In [6]:
moleculePath = "../1hpv.pdb"

atoms_df, hetatms_df = split_file(moleculePath)

for residue_name, group_hetatm in hetatms_df:
    print(group_hetatm.shape)
    if group_hetatm.empty:
        continue
    
    mol_with_hydrogens = add_hydrogens(group_hetatm)
    save_mol_as_pdb(mol_with_hydrogens, f"{residue_name}_hs.pdb")
    
    # print(hetatm_with_h)
    
    # if group_hetatm.empty:
    #     print(f"No HETATM records found for residue: {residue_name}")
    #     continue  # Skip empty groups
    
    # print(f"Processing HETATM records for residue: {residue_name}")
    # hetatm_withH = add_hydrogens(group_hetatm)
    # print(hetatm_withH.head())

(35, 21)
